In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from astropy.io import fits

%matplotlib inline

### Constants

In [13]:
t_max = 1.0e2  # in Myr
bin_width = 5.0  # arcseconds
bin_area = bin_width*bin_width  # square arcseconds

### Load Halpha Image

In [31]:
hdulist = fits.open("../data/NGC_4449_I_Ha_d2009_small.fits")
Halpha_map = hdulist[0].data
Halpha_map += np.abs(np.min(Halpha_map))
Halpha_map_err = np.sqrt(Halpha_map)

### Load global star formation history

Figure 6 from Cignoni et al. (2018)

In [14]:
t0, t1, t2, t3, t4, t5, t6 = 5.0, 10.0, 15.0, 25.0, 40.0, 60.0, 100.0  # Myr
sfr0, sfr1, sfr2, sfr3, sfr4, sfr5, sfr6, sfh7 = 0.018, 0.016, 0.032, 0.024, 0.025, 0.019, 0.028, 0.012  # Msun / yr 
sfr_err0, sfr_err1, sfr_err2, sfr_err3, sfr_err4, sfr_err5, sfr_err6, sfr_err7 = 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025


def get_sfr_global(t, c):
    
    SFH_global = np.piecewise(t, 
                             [t<t0, (t>t0)&(t<=t1), (t>t1)&(t<=t2), (t>t2)&(t<=t3), (t>t3)&(t<=t4), (t>t4)&(t<=t5), (t>t5)&(t<=t6), t>t6],
                             [c[0]*sfr0, c[1]*sfr1, c[2]*sfr2, c[3]*sfr3, c[4]*sfr4, c[5]*sfr5, c[6]*sfr6, c[7]*sfr7])

    return SFH_global


def get_sfr_err_global(t):
    
    SFH_err_global = np.piecewise(t, 
                                 [t<t0, (t>t0)&(t<=t1), (t>t1)&(t<=t2), (t>t2)&(t<=t3), (t>t3)&(t<=t4), (t>t4)&(t<=t5), (t>t5)&(t<=t6), t>t6],
                                 [sfr_err0, sfr_err1, sfr_err2, sfr_err3, sfr_err4, sfr_err5, sfr_err6, sfr_err7])

    return SFH_err_global

### Functions for model

In [33]:
def response_Halpha(t):
    return np.exp(-t/10.0)



def SFH_pixel(t, c):
    return get_sfr_global(t, c)


def integrand_I_Halpha(t, x, y, c):
    return response_Halpha(t) * SFH_pixel(t, c)
    

def calc_I_Halpha(x, y, c):
    return quad(integrand_I_Halpha, 0, t_max, args=(x,y,c))[0]





def ln_likelihood(c_full, Halpha_map):
    
    lnL_1 = calc_ln_likelihood_I_Halpha(c_full, Halpha_map)
    lnL_2 = calc_ln_likelihood_SFH_global(c_full)
    
    return lnL_1 + lnL_2


def calc_ln_likelihood_I_Halpha(c_full, Halpha_map):
    
    lnL = 0.0
    
    for i, x in enumerate(x_pixels):
        for j, y in enumerate(y_pixels):
            lnL += -(calc_I_Halpha(x, y, c) - Halpha_map[x,y])**2/Halpha_map_err[x,y]**2
            
    return lnL


def calc_ln_likelihood_SFH_global(c_full):
    
    lnL = 0.0
    
    t_grid = [2.5, 8.0, 12.0, 18.0, 28.0, 50.0, 75.0, 150.0]
    
    for k, t in enumerate(t_grid):
        
        sfh_integrated = 0.0
        for i, x in enumerate(x_pixels):
            for j, y in enumerate(y_pixels):
                sfh_integrated += get_sfr_global(t, c_full[i,j]) * bin_area
                
        lnL += -(sfh_integrated - sfh_global(t, np.ones(8).tolist()))**2/sfh_err_global(t)**2
            
    return lnL

### Initialize model parameters

In [ ]:
c_full = np.zeros((13, 13, 8))

for i, x in enumerate(x_pixels):
    for j, y in enumerate(y_pixels):
        c_full[i,j] = Halpha_map[i,j]/np.sum(Halpha_map) * np.ones(8)